<a href="https://colab.research.google.com/github/datdiego/qubit_dyamics/blob/main/notebooks/SingleQubiNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
# Load data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/quantum/dataset/fields_500_opt.csv")

In [ ]:
# Normalize inputs
df['theta'] /= (2 * np.pi)  # Normalize θ to [0, 1]
df['phi'] /= (2 * np.pi)    # Normalize φ to [0, 1]

# Prepare inputs and outputs
inputs = df[['theta', 'phi', 'time']].values
outputs = df[['u_x', 'u_y', 'u_z', 'sigmax_expect', 'sigmay_expect', 'sigmaz_expect']].values

t_list = np.linspace(0, 1, 100)  # Time grid over 1 second

In [ ]:
class PINNDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.outputs = torch.tensor(outputs, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

# Create dataset and dataloader
dataset = PINNDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
class QubiNN(nn.Module):
    def __init__(self):
        super(QubiNN, self).__init__()
        self.fc1 = nn.Linear(3, 128)  # Input: θ, φ, t
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc_out = nn.Linear(64, 6)  # Output: u_x, u_y, u_z, ⟨σx⟩, ⟨σy⟩, ⟨σz⟩

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc_out(x)
        return x




In [ ]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QubiNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs_batch, targets_batch in dataloader:
        # Move data to device
        inputs_batch, targets_batch = inputs_batch.to(device), targets_batch.to(device)

        # Forward pass
        predictions = model(inputs_batch)  # Shape: (batch_size, 6)

        # Compute data loss
        data_loss = criterion(predictions, targets_batch)

        # Backpropagation and optimization
        optimizer.zero_grad()
        data_loss.backward()
        optimizer.step()

        running_loss += data_loss.item() * inputs_batch.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Data Loss: {epoch_loss:.6f}")


In [ ]:
# Validation
model.eval()
with torch.no_grad():
    for inputs_batch, targets_batch in dataloader:
        inputs_batch, targets_batch = inputs_batch.to(device), targets_batch.to(device)
        predictions = model(inputs_batch)
        print("Predictions (first batch):", predictions[:5].cpu().numpy())
        print("Targets (first batch):", targets_batch[:5].cpu().numpy())
        break


In [ ]:
!pip install qutip
!pip install qutip-qtrl
!pip install qutip-qip

In [ ]:
import matplotlib.pyplot as plt
import time
import numpy as np
from numpy import pi
import qutip as qt
import pandas as pd
from qutip import *
from qutip_qtrl import *
from qutip_qip.operations import *
from numpy import pi
from qutip_qtrl.grape import plot_grape_control_fields, _overlap
from qutip_qtrl.cy_grape import cy_overlap
from qutip_qtrl.grape import cy_grape_unitary, grape_unitary_adaptive
from scipy.interpolate import interp1d
from qutip.ui.progressbar import TextProgressBar

In [ ]:
start_time = time.time()
# Define parameters
num_time_slices = 100  # Number of time slices for each GRAPE run
evo_time = 1.0  # Total evolution time
t_list = np.linspace(0, evo_time, num_time_slices)  # Time range
eps = 2 * np.pi / evo_time  # Epsilon scaling for GRAPE

# Define Hamiltonian components
H0 = 0 * np.pi * sigmaz()  # Drift Hamiltonian set to zero
H_ops = [sigmax(), sigmay(), sigmaz()]  # Control operators for x, y, and z directions
R =150

# Define the initial state (always starting from |0⟩)
psi0 = basis(2, 0)


#theta_test = np.pi
#phi_test = 0
# random angles
theta_test = np.random.uniform(0, 2 * np.pi)
phi_test = np.random.uniform(0, 2 * np.pi)


t_list = np.linspace(0, 1, 100)  # Time grid over 1 second

U = rz(phi_test) * rx(theta_test); U

# Initial guess for control fields with smoothing
u0 = np.array([np.random.rand(len(t_list)) * 2 * np.pi * 0.005 for _ in range(len(H_ops))])
u0 = [np.convolve(np.ones(10)/10, u0[idx, :], mode='same') for idx in range(len(H_ops))]

# Run cy_grape_unitary to find the optimal control fields for reaching the target unitary
result = cy_grape_unitary(
    U, H0, H_ops, R, t_list, u_start=u0, eps=eps,
        phase_sensitive=False
    )

# Solve for dynamics to get expectation values ⟨σx⟩, ⟨σy⟩, and ⟨σz⟩ over time
e_ops = [sigmax(), sigmay(), sigmaz()]
me_result = mesolve(result.H_t, psi0, t_list, c_ops=[], e_ops=e_ops)

qutip_execution_time = time.time() - start_time
print(f"QuTiP execution time: {qutip_execution_time} seconds")

In [ ]:
b = Bloch()
b.add_states(psi0)
b.add_states(U*psi0)
b.add_points(me_result.expect)
b.show()

In [ ]:
# Extract QuTiP control fields and expectation values
last_iteration_u = result.u[-1]
qutip_control_fields = last_iteration_u   # Shape: (150, 3, 100)
qutip_expectations = np.array(me_result.expect)  # Shape: (3, 100)

In [ ]:
print("Shape of QuTiP control fields:", qutip_control_fields.shape) # (3, 100)
print("Shape of QuTiP expectation values:", qutip_expectations.shape) # (3, 100)

In [ ]:
start_time = time.time()

# Prepare inputs for the PINN
theta_test_normalized = theta_test / (2 * np.pi)  # Normalize θ
phi_test_normalized = phi_test / (2 * np.pi)      # Normalize φ
pinn_inputs = np.array([[theta_test_normalized, phi_test_normalized, t] for t in t_list], dtype=np.float32)

# Convert inputs to PyTorch tensor
pinn_inputs_tensor = torch.tensor(pinn_inputs, device=device)

# Predict using the PINN
model.eval()
with torch.no_grad():
    pinn_outputs = model(pinn_inputs_tensor).cpu().numpy()  # Shape: (100, 6)

pinn_execution_time = time.time() - start_time
print(f"PINN execution time: {pinn_execution_time} seconds")

In [ ]:
# Calculate speedup
speedup = qutip_execution_time / pinn_execution_time

# Print speedup
print(f"Speedup of PINN over QuTiP Simulation: {speedup:.2f}x")

In [ ]:

# Extract control fields and expectation values
pinn_control_fields = pinn_outputs[:, :3].T  # u_x, u_y, u_z (Shape: (3, 100))
pinn_expectations = pinn_outputs[:, 3:].T    # ⟨σx⟩, ⟨σy⟩, ⟨σz⟩ (Shape: (3, 100))

In [ ]:
import matplotlib.pyplot as plt

expectations = ['⟨σx⟩', '⟨σy⟩', '⟨σz⟩']
for i, label in enumerate(expectations):
    plt.figure(figsize=(8, 6))
    plt.plot(t_list, qutip_expectations[i], label=f'QuTiP {label}', linestyle='--')
    plt.plot(t_list, pinn_expectations[i], label=f'PINN {label}', linestyle='-')
    plt.xlabel('Time (s)')
    plt.ylabel(label)
    plt.title(f'Comparison of {label} Over Time')
    plt.legend()
    plt.show()


In [ ]:
fields = ['u_x', 'u_y', 'u_z']
for i, field in enumerate(fields):
    plt.figure(figsize=(8, 6))
    plt.plot(t_list, qutip_control_fields[i], label=f'QuTiP {field}', linestyle='--')
    plt.plot(t_list, pinn_control_fields[i], label=f'PINN {field}', linestyle='-')
    plt.xlabel('Time (s)')
    plt.ylabel(field)
    # set y limit
    plt.ylim(-2 , 2)

    plt.title(f'Comparison of {field} Over Time')
    plt.legend()
    plt.show()


In [ ]:
# Create Bloch sphere for QuTiP data
b_qutip = Bloch()
b_qutip.add_points([qutip_expectations[0], qutip_expectations[1], qutip_expectations[2]], meth='l')
b_qutip.show()

In [ ]:
b_qutip.add_points([pinn_expectations[0], pinn_expectations[1], pinn_expectations[2]], meth='l')
b_qutip.show()

In [ ]:
from sklearn.metrics import mean_squared_error

# MSE for expectation values
mse_sigmax = mean_squared_error(qutip_expectations[0], pinn_expectations[0])
mse_sigmay = mean_squared_error(qutip_expectations[1], pinn_expectations[1])
mse_sigmaz = mean_squared_error(qutip_expectations[2], pinn_expectations[2])

print(f"MSE for ⟨σx⟩: {mse_sigmax:.6f}")
print(f"MSE for ⟨σy⟩: {mse_sigmay:.6f}")
print(f"MSE for ⟨σz⟩: {mse_sigmaz:.6f}")

# MSE for control fields
mse_ux = mean_squared_error(qutip_control_fields[0], pinn_control_fields[0])
mse_uy = mean_squared_error(qutip_control_fields[1], pinn_control_fields[1])
mse_uz = mean_squared_error(qutip_control_fields[2], pinn_control_fields[2])

print(f"MSE for u_x: {mse_ux:.6f}")
print(f"MSE for u_y: {mse_uy:.6f}")
print(f"MSE for u_z: {mse_uz:.6f}")


In [ ]:
# save model
torch.save(model.state_dict(), 'qubi_nn_model_11-15.pth')

In [ ]:
def calculate_density_matrix(sigma_x, sigma_y, sigma_z):
    """
    Calculate the density matrix from expectation values.

    Parameters:
        sigma_x (float): Expectation value of ⟨σx⟩.
        sigma_y (float): Expectation value of ⟨σy⟩.
        sigma_z (float): Expectation value of ⟨σz⟩.

    Returns:
        numpy.ndarray: The density matrix (2x2).
    """
    rho = np.array([
        [1 + sigma_z, sigma_x - 1j * sigma_y],
        [sigma_x + 1j * sigma_y, 1 - sigma_z]
    ]) / 2
    return rho

In [ ]:
# calculate rho of the last expected values of the pinn
rho = calculate_density_matrix(pinn_expectations[0][-1], pinn_expectations[1][-1], pinn_expectations[2][-1])

# convert to qutip object
rho_qutip = Qobj(rho)

In [ ]:
f = fidelity(rho_qutip, U*psi0)
print(f)